# Data Exploration

This notebook explores the all data processing steps in order from cleaning the raw data to generating the evaluation data template.

We start with two csv files. One contains information about cocoa producers contained in a pdf released by the govenrmnet of Ivory Coast. The other contains information about cocoa producers collected from large cocoa importers (e.g. Ferrero, Olam, Nestle, ...). In order to merge the datasets, we need to find all possible row combinations between the CSVs which describe the same entity. <br>

In [ ]:
import pandas as pd

In [ ]:
# Read CSV files into DataFrames
cooperatives_1 = pd.read_csv('data/raw/ivorian-cocoa-coop-registry-2017.csv')
cooperatives_2 = pd.read_csv('data/raw/cocoa-suppliers-compiled-from-importers.csv')

## Quick Look at Raw Data

In [ ]:
cooperatives_1

In [ ]:
cooperatives_2

## Missingness

In [ ]:
from vis import plot_missingness

# Plot missingness for each DataFrame
print(f'cooperatives 1: {cooperatives_1.shape}')
print(f'cooperatives 2: {cooperatives_2.shape}')
plot_missingness(cooperatives_1, 'cooperatives_1')
plot_missingness(cooperatives_2, 'cooperatives_2')

# Calculate and print percentage of missing values for each DataFrame
for name, df in [('cooperatives_1', cooperatives_1), ('cooperatives_2', cooperatives_2)]:
    print(f"\nPercentage of missing values in {name}:")
    print(df.isnull().sum() / len(df) * 100)

## Clean

When cleaning, we first pass over the govrnment data specifically, then use a different function to clean both government and company data, `cooperatives_1` and `cooperatives_2` respectively. 

Government data only: 
1. Remove rows that contain only null values
2. Rename columns to more descriptive names
3. Extract the abbreviation (text inside parentheses) from 'Producer Name' and create a new column for it
4. Remove the abbreviation text along with the parentheses from 'Producer Name'

Process both: 
1. Remove rows where 'Producer Name' or 'Abbreviation Name' are None
2. Clean the 'Producer Name' and 'Abbreviation Name' columns by forcing lower case, removing extra whitespace, normalizing unicode
5. Drop the 'Country' column if it exists

In [ ]:
cooperatives_1 = pd.read_csv('data/clean/ivorian-cocoa-coop-registry-2017.csv')
cooperatives_2 = pd.read_csv('data/clean/cocoa-suppliers-compiled-from-importers.csv')

# Plot missingness for each DataFrame
print(f'cooperatives 1: {cooperatives_1.shape}')
print(f'cooperatives 2: {cooperatives_2.shape}')
plot_missingness(cooperatives_1, 'cooperatives_1')
plot_missingness(cooperatives_2, 'cooperatives_2')

# Calculate and print percentage of missing values for each DataFrame
for name, df in [('cooperatives_1', cooperatives_1), ('cooperatives_2', cooperatives_2)]:
    print(f"\nPercentage of missing values in {name}:")
    print(df.isnull().sum() / len(df) * 100)

As you can see, there are some null values left in `Region`, but it will not matter because it is not used in evaluation. 

## Generate Name Pairs

Name pairs are generated by taking the cartesian product between the two cleaned CSVs.

In [ ]:
pairs = pd.read_csv('data/outputs/all_pairs.csv')
pairs

## Vector Silarity Tests

We use `paraphrase-MiniLM-L6-v2` from huggingface and TF-IDF vectorizer to embed the producer names and find each pair's cosine similarity. A high similarity should indicate a higher likelihood that the names represent the same entity. We define thresholds for each method and use the results to generate an eval set. These samples with high similarity will be more difficult to classify on average, and, thus, provide a good starting point for the eval set. 

In [ ]:
processed_pairs = pd.read_csv('data/outputs/pair_similarity.csv')
processed_pairs.describe()

In [ ]:
from vis import similarity_hist

similarity_hist(
    df=processed_pairs, 
    column='tf_idf_similarity_name', 
    title='Distribution of TF-IDF Similarity Scores', 
    threshold= 0.85
)

similarity_hist(
    df=processed_pairs, 
    column='semantic_similarity_name', 
    title='Distribution of Semantic Similarity Scores Between Producer Names', 
    threshold= 0.85
)

similarity_hist(
    df=processed_pairs, 
    column='second_half_similarity_name', 
    title='Distribution of Semantic Similarity Scores Between Second Half of Producer Names', 
    threshold= 0.85
)

similarity_hist(
    df=processed_pairs, 
    column='second_half_weighted_similarity', 
    title='Distribution of Semantic Similarity Scores Weighing the Second Half More', 
    threshold= 0.85
)

### Explore Pairs Ranked

Change `column` to the column from `processed_pairs` that you want to sort by to explore.

Your choices:
1. `tf_idf_similarity_name`
2. `semantic_similarity_name`
3. `second_half_similarity_name`
4. `second_half_weighted_similarity`

In [ ]:
column = 'semantic_similarity_name'
processed_pairs = processed_pairs.sort_values(by=column, ascending=False)
        
for i in range(100):
    row = processed_pairs.iloc[i]
    print(f"{row['Producer Name_x']:<50} | {row['Producer Name_y']:<50} | {row[column]:.4f} | {row['Abbreviation Name_x']:<10} | {row['Abbreviation Name_y']:<10} ")
    

One thing we can notice immediately from `semantic_similarity_name` is that the way the string starts seems to heavily sway the similarity resulting similarity measure. This is why we create `second_half_similarity_name` and combine the aforementioned two to create `second_half_weighted_similarity`.

## Evaluation Template

We generate the evaluation data template by pulling the top 200 pairs ranked by `second_half_weighted_similarity` and `tf_idf_similarity_name`. The duplicate rows are removed and new column `classification` is added with all calsses set to 0 (false).

In [ ]:
eval_template = pd.read_csv('data/outputs/eval_template.csv')
eval_template